**Framework version January 2024**
- Log :
    - understood bug in data (the problem was in CRAB not here)
    - added Snapshot 
- Planned update :
    - standalone code, prepare a couple of input parameters and make the code working with a single command
    
    
__________________________________________

**Code**

Folder definition on Tier:
- in the main folder */acagnott/* added folder 'remote_folder_name';
- in "Snapshots" added the folder 'remote_subfolder_name';
- in the subfolder through dask the snapshot will be copied with name "snap_"+label+"_*.root"


Es: se lancio "DataMETA_2018", gli snapshot vengono salvati in ../acagnott/Snapshot/20231229/snap_DataMET_2018_*.root
se viene lanciato "QCD_2018" viene creata la cartella /acagnott/Snapshot/20231229/snap_QCDHT_100to200_2018_*.root e così via per ogni components

---> Viene usato solo il giorno in modo che tutti i sample lanciati lo stesso giorno verranno salvati nella stessa cartella con nomi diversi, visto che vengono lanciati in momenti diversi della giornata lo stesso tipo di job. Forse va modificato il formato se i singoli job iniziano a durare più di un giorno dato che viene comunque lanciato un sampel per volta (in tal caso verrebbero salvati in cartelle diverse. Si potrebbe pensare di mettere la data a mano, cioé invece di usare datetime.now() si potrebbe inserire la data manualmente per fare in modo di mettere tutti i file nella stessa folder. 

In [1]:
from datetime import datetime
sched_port = 22703 #Dask port
nmaxpartition = 100 
distributed = True#False#
do_histos = True
hist_folder = "run2018_regions_v0"
do_snapshot = False
remote_subfolder_name = datetime.now().strftime("%Y%m%d") #20231229

In [2]:
import ROOT
import os
from utils.samples import *
from utils.variables import *
import json
import numpy as np
import math
from dask.distributed import Client
ROOT.RDF.Experimental.Distributed.open_files_locally = False

os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
os.environ['X509_USER_PROXY'] = "/tmp/x509up_u0"
print(os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR"))


if distributed:
    nfiles_max = 1000
else:
    nfiles_max = 1  #######

# Cosa aggiungere, modificare la cartella sul tier in questo modo ../acagnott/Snapshot_rdf/*dataset_name*/*data di processamente con orario*/
# insomma come fa crab


if do_histos: print("You are producing histograms")
if do_snapshot: print("You are producing snapshot")

remote_folder_name = "Snapshots"
#output histos folder
folder = "./results/"+hist_folder+"/"
# eos_folder = "/eos/home-a/acagnott/DarkMatter/nosynch/"+hist_folder

if do_snapshot and remote_subfolder_name == datetime.now().strftime("%Y%m%d") and distributed: 
    print("You are naming the tier subfolder using the current day \n")
    print("Snapshots folder name : ~/acagnott/{}/{}".format(remote_folder_name, remote_subfolder_name))
elif do_snapshot and distributed:
    print("You are naming the tier subfolder manually")
    print("Snapshots folder name : ~/acagnott/{}/{}".format(remote_folder_name, remote_subfolder_name))
elif do_snapshot:
    print("You are saving snapshots in local")
    print("folder name : " + folder)


if do_histos : 
    print("local folder histos: {}".format(folder))
    # print("At the end of the job the histos will be transfer to eos storage")
    # print("eos folder : {}".format(eos_folder))
if not os.path.exists(folder):
    os.mkdir(folder)
repohisto = folder+"plots/"
if not os.path.exists(repohisto):
    os.mkdir(repohisto)

Welcome to JupyROOT 6.27/01
/tmp/x509up_u0 /cvmfs/grid.cern.ch/etc/grid-security/certificates/
You are producing histograms
local folder histos: ./results/run2018_regions_v0/


In [3]:
in_dataset = [    
    # "DataMETA_2018", 
    # "DataMETB_2018", 
    # "DataMETC_2018",
    # "DataMETD_2018",
    # "DataSingleMuA_2018", 
    # "DataSingleMuB_2018", 
    # "DataSingleMuC_2018",
    # "DataSingleMuD_2018",
    
    # "TprimeToTZ_700_2018", 
    # "TprimeToTZ_1000_2018", 
    # "TprimeToTZ_1800_2018",
    
    # "QCD_2018",
    "TT_2018",
    # "ZJetsToNuNu_2018",
    # "WJets_2018",
    
    # "QCDHT_100to200_2018", "QCDHT_200to300_2018","QCDHT_300to500_2018", "QCDHT_500to700_2018", "QCDHT_700to1000_2018","QCDHT_1000to1500_2018", "QCDHT_1500to2000_2018","QCDHT_2000toInf_2018","TT_hadr_2018", "TT_semilep_2018", "TT_Mtt700to1000_2018", "TT_Mtt1000toInf_2018","ZJetsToNuNu_HT100to200_2018", "ZJetsToNuNu_HT200to400_2018", "ZJetsToNuNu_HT400to600_2018", "ZJetsToNuNu_HT600to800_2018", "ZJetsToNuNu_HT800to1200_2018", "ZJetsToNuNu_HT1200to2500_2018", "ZJetsToNuNu_HT2500toInf_2018""WJetsHT100to200_2018", "WJetsHT200to400_2018", "WJetsHT400to600_2018", "WJetsHT600to800_2018", "WJetsHT800to1200_2018", "WJetsHT1200to2500_2018", "WJetsHT2500toInf_2018"
]

branches = {"event", "run", "HT_eventHT",  
            "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60", "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight", 
            "HLT_Ele32_WPTight_Gsf", "HLT_Ele115_CaloIdVT_GsfTrkIdT", "HLT_Photon200", 
            "HLT_IsoMu24","HLT_PFMET120_PFMHT120_IDTight", "HLT_PFMETNoMu140_PFMHTNoMu140_IDTight", 
            "HLT_PFMET140_PFMHT140_IDTight", "MET_pt", "nVetoMuon", "nVetoElectron", "nJetBtag", 
            "nGoodJet", "nTightElectron", "nTightMuon", "MT"}

In [4]:
# Creating folders on Tier
if do_snapshot and distributed:
    tier_main_folder = "davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/"
    os.popen("davix-mkdir davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{} -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name))
    os.popen("davix-mkdir davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{} -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name, remote_subfolder_name))
    
# transfer function for dask worker
def transfer_to_tier(dask_worker):
    import os
    os.popen('for filename in snap_*.root; do davix-put $filename davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{}/$filename -E ./proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/; done'.format(remote_folder_name, remote_subfolder_name))
    return True #, os.popen("for filename in ./test_*.root; do echo $filename; done").read()

- Import of utils from variables.py
Cut (if any), Regions, Variables

- syncro between in_dataset and sample_dict (from sample.py) to syncronize labels and ather featurs of the dataset (as sigma if needed)
- import of samples_dict.json to load files list (path to reach them on tier)


In [5]:
cut = requirements # ---> see variables.py

regions_def = regions # ---> see variables.py
print("Regions to book: ")
for r in regions_def.keys():
    print("  "+r)
    
sample_file = open("utils/dict_samples.json", "rb")
samples = json.load(sample_file)
sample_file.close()

var = vars  # ---> variables.py
var2d = vars2D 

print("Variables for histograms :")
print([v._name for v in var])

datasets = []
for in_d in in_dataset:
    if not in_d in sample_dict.keys():
        print("Check the in_dataset string... ", sample_dict.keys())
    else : 
        datasets.append(sample_dict[in_d])
print("Datasets to process : ", [d.label for d in datasets])


chain = {}
ntot_events = {}
for d in datasets:
    if hasattr(d, "components"):
        samples_list = d.components
    else:
        samples_list = [d]
    chain[d.label] = {}
    ntot_events[d.label] = {}
    for s in samples_list:
        if distributed: 
            nfiles = len(samples[d.label][s.label]['strings'])
            for i, string in enumerate(samples[d.label][s.label]['strings']): 
                samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "root://stormgf2.pi.infn.it/")
            chain[d.label][s.label] = samples[d.label][s.label]['strings']
        else: 
            nfiles = nfiles_max
            chain[d.label][s.label] = samples[d.label][s.label]['strings'][:nfiles]
        ntot_events[d.label][s.label] = np.sum(samples[d.label][s.label]['ntot'][:nfiles])
        print("Dataset : "+s.label)
        print("# of files to process : ", nfiles)
        if distributed:
            print("files strings :\n  {}\n  {}\n  ... \n  {}\n  {}".format(chain[d.label][s.label][0], chain[d.label][s.label][1], chain[d.label][s.label][-2], chain[d.label][s.label][-1]))
        else :
            print("files strings :\n  {}".format(chain[d.label][s.label][0]))
        print("# of total events in the files to process (MC only, if Data the number is None) : ", ntot_events[d.label][s.label])

Regions to book: 
  Presel
  AH
  SL
  SEl
  SMu
  AH1lWR
  AH1lWREl
  AH1lWRMu
Variables for histograms :
['MET_pt', 'MET_phi', 'PuppiMET_pt', 'LeadingJetPt_pt', 'LeadingFatJetPt_pt', 'LeadingMuonPt_pt', 'LeadingElectronPt_pt', 'nJet', 'nJetBtag', 'nFatJet', 'MinDelta_phi', 'MaxEta_jet', 'HT_eventHT', 'PV_npvsGood']
Datasets to process :  ['TT_2018']
Dataset : TT_hadr_2018
# of files to process :  826
files strings :
  root://stormgf2.pi.infn.it//store/user/acagnott/DM_Run3_v0/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/TT_hadr_2018/231222_110836/0000/tree_hadd_755.root

  root://stormgf2.pi.infn.it//store/user/acagnott/DM_Run3_v0/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/TT_hadr_2018/231222_110836/0000/tree_hadd_378.root

  ... 
  root://stormgf2.pi.infn.it//store/user/acagnott/DM_Run3_v0/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/TT_hadr_2018/231222_110836/0000/tree_hadd_135.root

  root://stormgf2.pi.infn.it//store/user/acagnott/DM_Run3_v0/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/TT_

In [6]:
# invLepveto = ""#"!" # per non invertirlo basta passare stinga vuota
# met_cut = 250
# mdphi_cut = 0
# HLT_filter = "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60 || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight || HLT_Ele32_WPTight_Gsf || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200 || HLT_IsoMu24"

In [7]:
# initialization of clusters

# upload the proxyfile to the Dask workers to make them able to access data on the grid 

from distributed.diagnostics.plugin import UploadFile
def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    proxy_name = 'x509up_u0'
    os.environ['X509_USER_PROXY'] = working_dir + '/' + proxy_name
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    shutil.copyfile(working_dir + '/' + proxy_name, working_dir + '/../../../proxy')    
    os.environ['EXTRA_CLING_ARGS'] = "-O2"
    return os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR")

text_file = open("utils/postselection.h", "r")
data = text_file.read()
def my_initialization_function():
    print(ROOT.gInterpreter.ProcessLine(".O"))
    ROOT.gInterpreter.Declare('{}'.format(data))
    print("end of initialization")

# set up everything properly
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    client.register_worker_plugin(UploadFile("/tmp/x509up_u0"))
    client.run(set_proxy)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

/usr/local/lib/python3.8/dist-packages/distributed/client.py:1128: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| msgpack | 1.0.3  | 1.0.2     | 1.0.3   |
| toolz   | 0.11.2 | 0.11.1    | 0.11.2  |
+---------+--------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


0
end of initialization


In [8]:
################### utils ###################
def cut_string(cut):
    return cut.replace(" ", "").replace("&&","_").replace(">","_g_").replace(".","_").replace("==","_e_")

################### preselection ###############
def preselection(df):
    df = df.Define("GoodJet_idx", "GetGoodJet(Jet_pt, Jet_eta, Jet_jetId)")
    df = df.Define("nGoodJet", "nGoodJet(GoodJet_idx)")
    
    # if 'leptonveto' in cut:
    #     df = df.Filter(invLepveto+"LepVeto(Electron_pt, Electron_eta, Electron_cutBased, Muon_pt, Muon_eta, Muon_looseId )", "Lepton Veto")
    #     if "&& leptonveto" in cut:
    #         c_ = cut.replace("&& leptonveto","")
    #     elif "leptonveto &&" in cut:
    #         c_ = cut.replace("leptonveto &&","")
    #     elif "leptonveto" in cut:
    #         c_ = cut.replace("leptonveto","")    
    # else: 
    #     df = df
    #     c_ = cut
    df = df.Define("nTightElectron", "nTightElectron(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("TightElectron_idx", "TightElectron_idx(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("nVetoElectron", "nVetoElectron(Electron_pt, Electron_cutBased)")
    df = df.Define("nTightMuon", "nTightMuon(Muon_pt, Muon_eta, Muon_tightId)")
    df = df.Define("TightMuon_idx", "TightMuon_idx(Muon_pt, Muon_eta, Muon_tightId)")
    df = df.Define("nVetoMuon", "nVetoMuon(Muon_pt, Muon_eta, Muon_looseId)")
    df = df.Define("Lepton_flavour", "Lepton_flavour(nTightElectron, nTightMuon)").Define("Lep_pt", "Lepton_var(Lepton_flavour, Electron_pt, TightElectron_idx, Muon_pt, TightMuon_idx)").Define("Lep_phi", "Lepton_var(Lepton_flavour, Electron_phi, TightElectron_idx, Muon_phi, TightMuon_idx)")
    df = df.Define("MT", "sqrt(2 * Lep_pt * MET_pt * (1 - cos(Lep_phi - MET_phi)))")
    
    # df = df.Filter("MET_pt>"+ str(met_cut), "MET_pt>"+ str(met_cut))
    # df = df.Filter("MinDelta_phi>"+ str(mdphi_cut), "MinDeltaPhi>"+ str(mdphi_cut))
    
    # df = df.Filter("atLeast1jet_setparams(Jet_pt, Jet_eta, Jet_mass, Jet_jetId, 30, 4, 0, 1)", "At_least_1Ak4")
    # df = df.Filter("atLeast1fatjet_setparams(FatJet_pt, FatJet_msoftdrop, FatJet_eta, FatJet_jetId, 200, 6, 40, 1)", "at_least_1Ak8")

    df = df.Define("LeadingJetPt_idx", "GetLeadingPtJet(Jet_pt)")
    df = df.Define("LeadingJetPt_pt", "GetLeadingJetVar(LeadingJetPt_idx, Jet_pt)")
    df = df.Define("LeadingJetPt_eta", "GetLeadingJetVar(LeadingJetPt_idx, Jet_eta)")
    df = df.Define("LeadingJetPt_phi", "GetLeadingJetVar(LeadingJetPt_idx, Jet_phi)")
    df = df.Define("LeadingJetPt_mass", "GetLeadingJetVar(LeadingJetPt_idx, Jet_mass)")
    df = df.Define("LeadingFatJetPt_idx", "GetLeadingPtJet(FatJet_pt)")
    df = df.Define("LeadingFatJetPt_pt", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_pt)")
    df = df.Define("LeadingFatJetPt_eta", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_eta)")
    df = df.Define("LeadingFatJetPt_phi", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_phi)")
    df = df.Define("LeadingFatJetPt_mass", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_mass)")
    df = df.Define("LeadingMuonPt_idx", "GetLeadingPtLep(Muon_pt, Muon_eta, Muon_looseId)")
    df = df.Define("LeadingMuonPt_pt", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_pt)")
    df = df.Define("LeadingMuonPt_eta", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_eta)")
    df = df.Define("LeadingMuonPt_phi", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_phi)")
    df = df.Define("LeadingElectronPt_idx", "GetLeadingPtLep(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("LeadingElectronPt_pt", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_pt)")
    df = df.Define("LeadingElectronPt_eta", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_eta)")
    df = df.Define("LeadingElectronPt_phi", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_phi)")
    
    df = df.Define("nForwardJet", "nForwardJet(GoodJet_idx, Jet_eta)")
    df = df.Define("nJetBtag", "njetbtag(GoodJet_idx, Jet_btagDeepFlavB)")
        
    return df

############### trigger selection #####################
def trigger_filter(df):
    df_trig = df.Filter("HLT_PFMET120_PFMHT120_IDTight || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight", "trigger")
    return df_trig

############### top selection ########################
def select_top(df):
    df_goodtopMer = df.Define("GoodTopMer_idx", "select_TopMer(FatJet_deepTag_TvsQCD, FatJet_pt, FatJet_eta, FatJet_phi)")
    # ritorna gli indici dei FatJet che superano la trs del Top Merged (no overlap)
    df_goodtopMix = df_goodtopMer.Define("GoodTopMix_idx", "select_TopMix(TopHighPt_score2, TopHighPt_pt, TopHighPt_eta, TopHighPt_phi)")
    # ritorna gli indici dei FatJet che superano la trs del Top Merged (no overlap)
    df_goodtopRes = df_goodtopMix.Define("GoodTopRes_idx", "select_TopRes(TopLowPt_scoreDNN, TopLowPt_pt, TopLowPt_eta, TopLowPt_phi)")
    # ritorna gli indici dei Fatche superano la trs del Top Merged (no overlap)
    df_topcategory = df_goodtopRes.Define("EventTopCategory", "select_TopCategory(GoodTopMer_idx, GoodTopMix_idx, GoodTopRes_idx)")
    # return:  0- no top sel, 1- top merged, 2- top mix, 3- top resolved
    df_topselected = df_topcategory.Define("Top_idx",
                                           "select_bestTop(EventTopCategory, GoodTopMer_idx, GoodTopMix_idx, GoodTopRes_idx, FatJet_deepTag_TvsQCD, TopHighPt_score2, TopLowPt_scoreDNN)")
    # return best top idx wrt category --> the idx is referred to the list of candidates fixed by the EventTopCategory
    df_topvariables = df_topselected.Define("Top_pt", "select_TopVar(EventTopCategory, Top_idx, FatJet_pt, TopHighPt_pt, TopLowPt_pt)")\
                        .Define("Top_eta", "select_TopVar(EventTopCategory, Top_idx, FatJet_eta, TopHighPt_eta, TopLowPt_eta)")\
                        .Define("Top_phi", "select_TopVar(EventTopCategory, Top_idx, FatJet_phi, TopHighPt_phi, TopLowPt_phi)")\
                        .Define("Top_mass", "select_TopVar(EventTopCategory, Top_idx, FatJet_mass, TopHighPt_mass, TopLowPt_mass)")\
                        .Define("Top_score", "select_TopVar(EventTopCategory, Top_idx, FatJet_deepTag_TvsQCD, TopHighPt_score2, TopLowPt_scoreDNN)")

    return df_topvariables

In [9]:
def bookhisto(df, regions_def, var, s_cut):
    h_ = {}
    for reg in regions_def.keys():
        h_[reg] = {}
        for v in var:
            if regions_def[reg] == "":
                h_[reg][v._name]= df.Redefine(v._name, "UnOvBin("+v._name+","+str(v._nbins)+","+str(v._xmin)+","+str(v._xmax)+")").Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name)
                # h_[reg][v._name]= df.Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name)
            else:
                h_[reg][v._name]= df.Filter(regions_def[reg]).Redefine(v._name, "UnOvBin("+v._name+","+str(v._nbins)+","+str(v._xmin)+","+str(v._xmax)+")").Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name)
                # h_[reg][v._name]= df.Filter(regions_def[reg]).Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name)            
    return h_

def bookhisto2D(df, regions_def, var2d, s_cut):
    h_ = {}
    for reg in regions_def.keys():
        h_[reg] = {}
        for v in var2d:
            if regions_def[reg]=="":
                h_[reg][v._name] = df.Redefine(v._xname, "UnOvBin("+v._xname+","+str(v._nxbins)+","+str(v._xmin)+","+str(v._xmax)+")")\
                                     .Redefine(v._yname, "UnOvBin("+v._yname+","+str(v._nybins)+","+str(v._ymin)+","+str(v._ymax)+")")\
                                     .Histo2D((v._xname+"Vs"+v._yname+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax), v._xname, v._yname)
            else:
                h_[reg][v._name] = df.Filter(regions_def[reg])\
                                     .Redefine(v._xname, "UnOvBin("+v._xname+","+str(v._nxbins)+","+str(v._xmin)+","+str(v._xmax)+")")\
                                     .Redefine(v._yname, "UnOvBin("+v._yname+","+str(v._nybins)+","+str(v._ymin)+","+str(v._ymax)+")")\
                                     .Histo2D((v._xname+"Vs"+v._yname+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax), v._xname, v._yname)
    return h_

In [10]:
def savehisto(d, h, regions_def, var, s_cut):
    histo = {reg: {v._name: ROOT.TH1D(v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax) for v in var} for reg in regions_def.keys()}
    
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]
    
    for s in s_list:
        outfile = ROOT.TFile.Open(repohisto+s.label+'.root', "RECREATE")
        for reg in regions_def.keys():
            for v in var:
                histo[reg][v._name] = h[d.label][s.label][reg][v._name].GetValue()      
                outfile.cd()
                histo[reg][v._name].Write()
        outfile.Close()

# i plot2d per il momento non ci servono, si deve trovare un modo più intelligente di farli
def savehisto2d(d, h, regions_def, var2d, s_cut):
    histo = {reg: {v._name: ROOT.TH2D(v._name+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax,) for v in var2d} for reg in regions_def.keys()}
        
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]
    
    for s in s_list:
        outfile = ROOT.TFile.Open(repohisto+s.label+'_2D.root', "RECREATE")
        for reg in regions_def.keys():
            for v in histo[reg].keys():
                histo[reg][v] = h[d.label][s.label][reg][v].GetValue()      
                outfile.cd()
                histo[reg][v].Write()
        outfile.Close()

In [11]:
#samples['DataHTF_2022']['DataHTF_2022']['strings']

In [12]:
#samples[d.label][d.components[0]]['strings']

In [ ]:
t0 = datetime.now()
print("starting loop on datasets: ",[d.label for d in datasets])
print("Local time :", t0)
# print("requirements: "+cut)

h = {}
h_2D = {}

for d in datasets:
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]

    if 'Data' in d.label : sampleflag = 0
    else: sampleflag = 1
    c_ = cut
    h[d.label] = {}
    h_2D[d.label] = {}
    for s in s_list:
        
        print("Initializing DataFrame for "+ s.label +" chain len = ", len(chain[d.label][s.label]))
        if len(chain[d.label][s.label])==1: print(chain[d.label][s.label])
        if distributed ==True:
            df = RDataFrame("Events", chain[d.label][s.label], npartitions=nmaxpartition, 
                            daskclient=client, monitor_label = "main" )
        else:
            df = RDataFrame("Events", chain[d.label][s.label])
        
        
        df_ismc         = df.Define("isMC", "isMC("+str(sampleflag)+")")
        df_year         = df_ismc.Define("year", str(s.year))
        df_hemveto      = df_year.Define("HEMVeto", "hemveto(Jet_eta, Jet_phi, Electron_eta, Electron_phi)")
        df_hemveto      = df_hemveto.Filter("(isMC || (year != 2018) || (HEMVeto || run<319077.))")
        df_hlt          = trigger_filter(df_hemveto)        
        ### per gli HLT potremmo volerli modificare a seconda della regione (?) in tal caso la funzione
        ### va spostata nel book histos e va passata la regione per attivare trigger diversi
        df_wnom         = df_hlt.Define('w_nominal', '1')
        df_presel       = preselection(df_wnom)
        
        if do_snapshot:
            opts = ROOT.RDF.RSnapshotOptions()
            opts.fLazy = True
            if distributed: fold = "./"
            else: fold = folder
            snapshot_df = df_presel.Snapshot("events_nominal", fold+"snap_"+s.label+".root", branches, opts)
            # print("./"+s.label+".root")
        if do_histos:
            s_cut = cut_string(cut)
            if len(var) != 0 :
                h[d.label][s.label] = bookhisto(df_presel, regions_def, var, s_cut)
            if len(var2d) != 0 :
                h_2D[d.label][s.label] = bookhisto2D(df_presel, regions_def, var2d, s_cut)
# if not distributed:
#     df_presel.Report().Print()

if do_histos:
    print("All histos booked !")
    for d in datasets:
        if len(var):
            savehisto(d, h, regions_def, var, s_cut)
        if len(var2d) != 0 :
            savehisto2d(d, h_2D, regions_def, var2d, s_cut)
        print(d.label + " histos saved")
if do_snapshot:
    snapshot_df.GetValue()
    if distributed: 
        client.run(transfer_to_tier)
        print("Snapshots saved and trasfered to tier")
    print("Sanpshot done!")
t1 = datetime.now()
print("Job finished in: ", t1-t0)

starting loop on datasets:  ['TT_2018']
Local time : 2024-01-04 17:29:32.432929
Initializing DataFrame for TT_hadr_2018 chain len =  826
Initializing DataFrame for TT_semilep_2018 chain len =  1029
Initializing DataFrame for TT_Mtt1000toInf_2018 chain len =  71
Initializing DataFrame for TT_Mtt700to1000_2018 chain len =  76
All histos booked !


In [ ]:
# file = ROOT.TFile.Open(repohisto+s.label+".root")
# for a in file.GetListOfKeys(): print(a)
# for reg in regions_def.keys():
#     hist = file.Get(var+"_"+reg+"_")
#     print(reg, hist.Integral())

In [ ]:
# regions_def.keys()

In [ ]:
# region = "NoCut"
# var = "MET_pt"

In [ ]:
if do_snapshot and distributed:
    print(os.popen("davix-ls davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{}/ -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name, remote_subfolder_name)).read())